In [32]:
import pandas as pd
import numpy as np
import os
from save_excel import save_excel
from openpyxl import load_workbook
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Import the cleaned dataset
csv_file_path = '..\\dataset\\international_top_terms_cleaned.csv'
df = pd.read_csv(csv_file_path)
df.head(10)

country_code region_name region_code        week  score  rank country_name  \
0           FR   Aquitaine        FR-B  2019-09-15   57.0    14       France   
1           FR    Auvergne        FR-C  2019-09-08   83.0    14       France   
2           FR    Burgundy        FR-D  2019-09-01  100.0    14       France   
3           FR    Burgundy        FR-D  2021-09-12   62.0    14       France   
4           FR    Burgundy        FR-D  2021-11-21   64.0    14       France   
5           FR    Burgundy        FR-D  2022-01-09   51.0    14       France   
6           FR    Burgundy        FR-D  2022-11-20   62.0    14       France   
7           FR    Burgundy        FR-D  2022-12-11   51.0    14       France   
8           FR    Burgundy        FR-D  2023-07-30   51.0    14       France   
9           FR    Burgundy        FR-D  2023-11-19   60.0    14       France   

    term refresh_date  
0  Apple   2024-06-14  
1  Apple   2024-06-14  
2  Apple   2024-06-14  
3  Apple   2024-06-14  
4  Apple   2024-06-14  
5  Apple   2024-06-14  
6  Apple   2024-06-14  
7  Apple   2024-06-14  
8  Apple   2024-06-14  
9  Apple   2024-06-14

# Splitting the Data in Accordance to the Country

## Australia
```country_name```

```region_name```

```term```

```score```

```mean``` - rows with same region_name and term will have their mean score computed

In [57]:
basket_Australia = (df[df['country_name'] == "Australia"]
          .groupby(['region_name', 'term'])['score'] 
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Australia)

term                          Alex de Minaur  Alexa Leary  Alice Springs  \
region_name                                                                
Australian Capital Territory       62.000000          0.0      57.000000   
New South Wales                    56.500000          0.0       0.000000   
Northern Territory                100.000000        100.0      77.589744   
Queensland                         75.333333          0.0      54.400000   
South Australia                    72.000000          0.0      64.214286   
Tasmania                           75.000000          0.0      61.000000   
Victoria                           59.000000          0.0      65.666667   
Western Australia                  82.000000          0.0      66.750000   

term                              Apple  Ash Barty  Bacchus Marsh Grammar  \
region_name                                                                 
Australian Capital Territory  65.750000        0.0                    0.0   
New Sout

### Conversion to binary vector

In [34]:
def hot_encode(x):
    if(x <= 0): 
        return 0
    if(x >= 1): 
        return 1

basket_encoded = basket_Australia.map(hot_encode) 
basket_Australia = basket_encoded

print(basket_Australia)

term                          Alex de Minaur  Alexa Leary  Alice Springs  \
region_name                                                                
Australian Capital Territory               1            0              1   
New South Wales                            1            0              0   
Northern Territory                         1            1              1   
Queensland                                 1            0              1   
South Australia                            1            0              1   
Tasmania                                   1            0              1   
Victoria                                   1            0              1   
Western Australia                          1            0              1   

term                          Apple  Ash Barty  Bacchus Marsh Grammar  \
region_name                                                             
Australian Capital Territory      1          0                      0   
New South Wales     

In [35]:
# Building the model 
frq_items = apriori(basket_Australia.astype('bool'), min_support = 0.9, use_colnames = True)
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

display(rules)
save_excel(rules, "Australia")

antecedents                                        consequents  \
0                (Apple)                                   (Alex de Minaur)   
1       (Alex de Minaur)                                            (Apple)   
2       (Alex de Minaur)                                 (Champions League)   
3     (Champions League)                                   (Alex de Minaur)   
4              (Cricket)                                   (Alex de Minaur)   
...                  ...                                                ...   
6045    (Premier League)  (UFC, Apple, Cricket, Alex de Minaur, Full moo...   
6046    (Alex de Minaur)  (UFC, Apple, Cricket, Premier League, Full moo...   
6047         (Full moon)  (UFC, Apple, Cricket, Premier League, Alex de ...   
6048               (NBA)  (UFC, Apple, Cricket, Premier League, Alex de ...   
6049  (Champions League)  (UFC, Apple, Cricket, Premier League, Alex de ...   

      antecedent support  consequent support  support  confidence  lift  \
0                    1.0                 1.0      1.0         1.0   1.0   
1                    1.0                 1.0      1.0         1.0   1.0   
2                    1.0                 1.0      1.0         1.0   1.0   
3                    1.0                 1.0      1.0         1.0   1.0   
4                    1.0                 1.0      1.0         1.0   1.0   
...                  ...                 ...      ...         ...   ...   
6045                 1.0                 1.0      1.0         1.0   1.0   
6046                 1.0                 1.0      1.0         1.0   1.0   
6047                 1.0                 1.0      1.0         1.0   1.0   
6048                 1.0                 1.0      1.0         1.0   1.0   
6049                 1.0                 1.0      1.0         1.0   1.0   

      leverage  conviction  zhangs_metric  
0          0.0         inf            0.0  
1          0.0         inf            0.0  
2          0.0         inf            0.0  
3          0.0         inf            0.0  
4          0.0         inf            0.0  
...        ...         ...            ...  
6045       0.0         inf            0.0  
6046       0.0         inf            0.0  
6047       0.0         inf            0.0  
6048       0.0         inf            0.0  
6049       0.0         inf            0.0  

[6050 rows x 10 columns]

## Argentina

In [36]:
# Top terms in Argentina
basket_Argentina = (df[df['country_name'] == "Argentina"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Argentina)

term                                Alcaraz    Aldosivi  Alphonso Davies  \
region_name                                                                
Autonomous City of Buenos Aires   75.000000   55.000000              0.0   
Buenos Aires Province             50.000000   71.500000              0.0   
Catamarca Province                 0.000000  100.000000              0.0   
Chaco Province                    59.000000    0.000000              0.0   
Chubut Province                   57.500000    0.000000              0.0   
Cordoba                            0.000000   64.333333              0.0   
Corrientes Province               89.000000   57.000000             57.0   
Entre Rios                       100.000000    0.000000             70.0   
Formosa Province                  75.000000    0.000000              0.0   
Jujuy                              0.000000    0.000000              0.0   
La Pampa Province                  0.000000   56.000000              0.0   
La Rioja Pro

In [37]:
basket_encoded_Argentina = basket_Argentina.map(hot_encode) 
basket_Argentina = basket_encoded_Argentina

In [38]:
# Building the model 
frq_items = apriori(basket_Argentina.astype('bool'), min_support = 0.5, use_colnames = True)
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Argentina")

antecedents  \
61                      (Bridgerton)   
88                    (Copa america)   
158            (Bridgerton, Alcaraz)   
536   (Bridgerton, Futbol argentino)   
548   (Bridgerton, Primera división)   
...                              ...   
4158                         (River)   
4249                         (River)   
4435                         (River)   
4498                         (River)   
4560                         (River)   

                                            consequents  antecedent support  \
61                                      (Independiente)            0.625000   
88                                        (San Antonio)            0.500000   
158                                     (Independiente)            0.500000   
536                                     (Independiente)            0.541667   
548                                     (Independiente)            0.541667   
...                                                 ...                 ...   
4158           (Independiente, Racing, TN, San Antonio)            1.000000   
4249  (Aldosivi, Futbol argentino, San Antonio, Raci...            1.000000   
4435  (Independiente, Futbol argentino, TN, Racing, ...            1.000000   
4498  (Independiente, Futbol argentino, TN, San Anto...            1.000000   
4560  (Independiente, Futbol argentino, TN, San Anto...            1.000000   

      consequent support   support  confidence      lift  leverage  \
61                 0.875  0.625000         1.0  1.142857  0.078125   
88                 0.875  0.500000         1.0  1.142857  0.062500   
158                0.875  0.500000         1.0  1.142857  0.062500   
536                0.875  0.541667         1.0  1.142857  0.067708   
548                0.875  0.541667         1.0  1.142857  0.067708   
...                  ...       ...         ...       ...       ...   
4158               0.500  0.500000         0.5  1.000000  0.000000   
4249               0.500  0.500000         0.5  1.000000  0.000000   
4435               0.500  0.500000         0.5  1.000000  0.000000   
4498               0.500  0.500000         0.5  1.000000  0.000000   
4560               0.500  0.500000         0.5  1.000000  0.000000   

      conviction  zhangs_metric  
61           inf       0.333333  
88           inf       0.250000  
158          inf       0.250000  
536          inf       0.272727  
548          inf       0.272727  
...          ...            ...  
4158         1.0       0.000000  
4249         1.0       0.000000  
4435         1.0       0.000000  
4498         1.0       0.000000  
4560         1.0       0.000000  

[4624 rows x 10 columns]

In [39]:
# Top terms in Philippines
basket_Philippines = (df[df['country_name'] == "Philippines"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Philippines)

term                                     19     Access  Angelina Jolie  \
region_name                                                              
Autonomous Region in Muslim Mindanao    0.0  59.750000        0.000000   
Bicol                                  67.0  65.137931        0.000000   
Cagayan Valley                          0.0  63.428571        0.000000   
Calabarzon                             72.0  70.625000        0.000000   
Caraga                                100.0  66.866667       92.000000   
Central Luzon                          96.0  67.862069        0.000000   
Central Visayas                        58.0  69.111111       59.000000   
Cordillera Administrative Region        0.0  69.312500      100.000000   
Davao Region                           56.5  65.172414       76.333333   
Eastern Visayas                        67.0  59.500000        0.000000   
Ilocos Region                           0.0  59.772727       76.000000   
MIMAROPA                              

In [40]:
basket_encoded_Philippines = basket_Philippines.map(hot_encode) 
basket_Philippines = basket_encoded_Philippines

print(basket_Philippines)

term                                  19  Access  Angelina Jolie  Anouk Aimée  \
region_name                                                                     
Autonomous Region in Muslim Mindanao   0       1               0            0   
Bicol                                  1       1               0            0   
Cagayan Valley                         0       1               0            0   
Calabarzon                             1       1               0            0   
Caraga                                 1       1               1            0   
Central Luzon                          1       1               0            0   
Central Visayas                        1       1               1            1   
Cordillera Administrative Region       0       1               1            0   
Davao Region                           1       1               1            0   
Eastern Visayas                        1       1               0            0   
Ilocos Region               

In [41]:
# Building the model 
frq_items = apriori(basket_Philippines.astype('bool'), min_support = 0.8, use_colnames = True)

print("After frequent Items")
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Philippines")

After frequent Items


antecedents                             consequents  \
0       (ESPN NBA)                                (Access)   
2     (Jose Rizal)                                (Access)   
4           (June)                                (Access)   
5         (Access)                                  (June)   
6            (NBA)                                (Access)   
...            ...                                     ...   
1441        (June)   (UFC, Access, Sports, NBA games, NBA)   
1442      (Access)     (UFC, June, Sports, NBA games, NBA)   
1443      (Sports)     (UFC, June, Access, NBA games, NBA)   
1444   (NBA games)        (UFC, June, Access, Sports, NBA)   
1445         (NBA)  (UFC, June, Access, Sports, NBA games)   

      antecedent support  consequent support   support  confidence  lift  \
0               0.882353            1.000000  0.882353    1.000000   1.0   
2               0.823529            1.000000  0.823529    1.000000   1.0   
4               1.000000            1.000000  1.000000    1.000000   1.0   
5               1.000000            1.000000  1.000000    1.000000   1.0   
6               1.000000            1.000000  1.000000    1.000000   1.0   
...                  ...                 ...       ...         ...   ...   
1441            1.000000            0.823529  0.823529    0.823529   1.0   
1442            1.000000            0.823529  0.823529    0.823529   1.0   
1443            1.000000            0.823529  0.823529    0.823529   1.0   
1444            1.000000            0.823529  0.823529    0.823529   1.0   
1445            1.000000            0.823529  0.823529    0.823529   1.0   

      leverage  conviction  zhangs_metric  
0          0.0         inf            0.0  
2          0.0         inf            0.0  
4          0.0         inf            0.0  
5          0.0         inf            0.0  
6          0.0         inf            0.0  
...        ...         ...            ...  
1441       0.0         1.0            0.0  
1442       0.0         1.0            0.0  
1443       0.0         1.0            0.0  
1444       0.0         1.0            0.0  
1445       0.0         1.0            0.0  

[1446 rows x 10 columns]

In [42]:
# Top terms in South Korea
basket_SouthKorea = (df[df['country_name'] == "South Korea"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_SouthKorea)

term                 CPI        NBA   WWDC    강민구    고준희    권경애    김계란    김레아  \
region_name                                                                     
Busan                0.0  63.333333    0.0    0.0    0.0    0.0   83.5    0.0   
Chungcheongbuk-do    0.0  63.666667    0.0    0.0    0.0    0.0    0.0   80.0   
Chungcheongnam-do   77.0  60.571429    0.0  100.0  100.0    0.0   56.0    0.0   
Daegu                0.0  59.000000    0.0    0.0    0.0    0.0   52.0   94.0   
Daejeon            100.0  55.500000    0.0    0.0    0.0  100.0    0.0    0.0   
Gangwon-do           0.0  54.000000    0.0    0.0    0.0    0.0  100.0    0.0   
Gwangju            100.0  62.000000    0.0    0.0    0.0    0.0   84.0  100.0   
Gyeonggi-do         77.5  61.571429    0.0  100.0    0.0    0.0   64.5    0.0   
Gyeongsangbuk-do     0.0  64.894737    0.0    0.0    0.0    0.0   61.5    0.0   
Gyeongsangnam-do     0.0  54.000000  100.0  100.0    0.0    0.0   67.0    0.0   
Incheon             59.0  54

In [43]:
basket_encoded_SouthKorea = basket_SouthKorea.map(hot_encode) 
basket_SouthKorea = basket_encoded_SouthKorea

print(basket_SouthKorea)

term               CPI  NBA  WWDC  강민구  고준희  권경애  김계란  김레아  김미성  노스트라다무스 예언  \
region_name                                                                   
Busan                0    1     0    0    0    0    1    0    0           0   
Chungcheongbuk-do    0    1     0    0    0    0    0    1    0           0   
Chungcheongnam-do    1    1     0    1    1    0    1    0    0           1   
Daegu                0    1     0    0    0    0    1    1    0           1   
Daejeon              1    1     0    0    0    1    0    0    0           0   
Gangwon-do           0    1     0    0    0    0    1    0    0           1   
Gwangju              1    1     0    0    0    0    1    1    0           0   
Gyeonggi-do          1    1     0    1    0    0    1    0    0           0   
Gyeongsangbuk-do     0    1     0    0    0    0    1    0    0           0   
Gyeongsangnam-do     0    1     1    1    0    0    1    0    0           0   
Incheon              1    1     0    0    1    0    

In [44]:
# Building the model 
frq_items = apriori(basket_SouthKorea.astype('bool'), min_support = 0.8, use_colnames = True)

print("After frequent Items")
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "South_Korea")

After frequent Items


antecedents                        consequents  antecedent support  \
0          (라이엇)                              (NBA)              0.9375   
2      (리그오브레전드)                              (NBA)              0.8750   
4       (메이플스토리)                              (NBA)              1.0000   
5          (NBA)                           (메이플스토리)              1.0000   
6         (엔비디아)                              (NBA)              1.0000   
..           ...                                ...                 ...   
595  (엔비디아, NBA)         (메이플스토리, 한국, 라이엇, 리그오브레전드)              1.0000   
596         (한국)  (라이엇, 메이플스토리, 리그오브레전드, 엔비디아, NBA)              1.0000   
598     (메이플스토리)      (한국, 라이엇, 리그오브레전드, 엔비디아, NBA)              1.0000   
600       (엔비디아)    (한국, 라이엇, 메이플스토리, 리그오브레전드, NBA)              1.0000   
601        (NBA)   (한국, 라이엇, 메이플스토리, 리그오브레전드, 엔비디아)              1.0000   

     consequent support  support  confidence  lift  leverage  conviction  \
0                1.0000   0.9375      1.0000   1.0       0.0         inf   
2                1.0000   0.8750      1.0000   1.0       0.0         inf   
4                1.0000   1.0000      1.0000   1.0       0.0         inf   
5                1.0000   1.0000      1.0000   1.0       0.0         inf   
6                1.0000   1.0000      1.0000   1.0       0.0         inf   
..                  ...      ...         ...   ...       ...         ...   
595              0.8125   0.8125      0.8125   1.0       0.0         1.0   
596              0.8125   0.8125      0.8125   1.0       0.0         1.0   
598              0.8125   0.8125      0.8125   1.0       0.0         1.0   
600              0.8125   0.8125      0.8125   1.0       0.0         1.0   
601              0.8125   0.8125      0.8125   1.0       0.0         1.0   

     zhangs_metric  
0              0.0  
2              0.0  
4              0.0  
5              0.0  
6              0.0  
..             ...  
595            0.0  
596            0.0  
598            0.0  
600            0.0  
601            0.0  

[602 rows x 10 columns]

In [45]:
# Top terms in Finland
basket_Finland = (df[df['country_name'] == "Finland"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Finland)

term                   Adam Lambert  Albania       Alko      Apple  Barkov  \
region_name                                                                  
Central Finland                 0.0    100.0  59.333333  59.500000     0.0   
Central Ostrobothnia            0.0      0.0  63.000000  58.000000     0.0   
Kainuu                        100.0     94.0  50.000000  82.500000     0.0   
Kymenlaakso                     0.0      0.0  52.500000  61.000000     0.0   
Lapland                       100.0      0.0  69.250000  57.333333     0.0   
North Karelia                   0.0    100.0  57.000000  59.200000     0.0   
Northern Ostrobothnia          76.0     88.0  73.000000  64.166667   100.0   
Northern Savonia                0.0     75.0   0.000000  62.100000     0.0   
Ostrobothnia                    0.0      0.0  62.000000  57.900000     0.0   
Pirkanmaa                       0.0      0.0  71.500000  54.166667     0.0   
Päijänne Tavastia               0.0      0.0  61.000000  53.7142

In [46]:
basket_encoded_Finland = basket_Finland.map(hot_encode) 
basket_Finland = basket_encoded_Finland

print(basket_Finland)

term                   Adam Lambert  Albania  Alko  Apple  Barkov  Bellingham  \
region_name                                                                     
Central Finland                   0        1     1      1       0           0   
Central Ostrobothnia              0        0     1      1       0           0   
Kainuu                            1        1     1      1       0           1   
Kymenlaakso                       0        0     1      1       0           1   
Lapland                           1        0     1      1       0           0   
North Karelia                     0        1     1      1       0           0   
Northern Ostrobothnia             1        1     1      1       1           0   
Northern Savonia                  0        1     0      1       0           0   
Ostrobothnia                      0        0     1      1       0           0   
Pirkanmaa                         0        0     1      1       0           0   
Päijänne Tavastia           

In [47]:
# Building the model 
frq_items = apriori(basket_Finland.astype('bool'), min_support = 0.3, use_colnames = True)
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Finland")

antecedents  \
1300               (Bridgerton, Billie Eilish)   
1647         (Bridgerton, Alko, Billie Eilish)   
1677        (Apple, Bridgerton, Billie Eilish)   
1686               (Bridgerton, Billie Eilish)   
1712  (Apple, Bridgerton, Alko, Billie Eilish)   
...                                        ...   
1542                                   (Apple)   
1573                                   (Apple)   
1603                                   (Apple)   
1634                                   (Apple)   
1755                                   (Apple)   

                                         consequents  antecedent support  \
1300                                   (UFC, Nvidia)            0.388889   
1647                                   (UFC, Nvidia)            0.333333   
1677                                   (UFC, Nvidia)            0.388889   
1686                            (UFC, Apple, Nvidia)            0.388889   
1712                                   (UFC, Nvidia)            0.333333   
...                                              ...                 ...   
1542            (Ronaldo, Nvidia, Alko, Real Madrid)            1.000000   
1573              (Star Wars, Ronaldo, Nvidia, Alko)            1.000000   
1603                    (UFC, Ronaldo, Nvidia, Alko)            1.000000   
1634                  (UFC, Star Wars, Nvidia, Alko)            1.000000   
1755  (UFC, Bridgerton, Alko, Billie Eilish, Nvidia)            1.000000   

      consequent support   support  confidence  lift  leverage  conviction  \
1300            0.500000  0.388889    1.000000   2.0  0.194444         inf   
1647            0.500000  0.333333    1.000000   2.0  0.166667         inf   
1677            0.500000  0.388889    1.000000   2.0  0.194444         inf   
1686            0.500000  0.388889    1.000000   2.0  0.194444         inf   
1712            0.500000  0.333333    1.000000   2.0  0.166667         inf   
...                  ...       ...         ...   ...       ...         ...   
1542            0.333333  0.333333    0.333333   1.0  0.000000         1.0   
1573            0.333333  0.333333    0.333333   1.0  0.000000         1.0   
1603            0.333333  0.333333    0.333333   1.0  0.000000         1.0   
1634            0.333333  0.333333    0.333333   1.0  0.000000         1.0   
1755            0.333333  0.333333    0.333333   1.0  0.000000         1.0   

      zhangs_metric  
1300       0.818182  
1647       0.750000  
1677       0.818182  
1686       0.818182  
1712       0.750000  
...             ...  
1542       0.000000  
1573       0.000000  
1603       0.000000  
1634       0.000000  
1755       0.000000  

[1758 rows x 10 columns]

In [48]:
# Top terms in United Kingdom
basket_UK = (df[df['country_name'] == "United Kingdom"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_UK)

term              Alan Shearer  Albania      Apple  BGT final  Bath Rugby  \
region_name                                                                 
England                    0.0      0.0  58.176471       74.0        50.0   
Northern Ireland          73.5      0.0  56.909091        0.0         0.0   
Scotland                   0.0      0.0  62.238095        0.0        59.5   
Wales                     58.0     74.0  56.444444        0.0        67.0   

term              Billings  Billy Harris  Bridgerton  Britain's Got Talent  \
region_name                                                                  
England                0.0         100.0         0.0                   0.0   
Northern Ireland      94.0           0.0         0.0                   0.0   
Scotland               0.0           0.0        61.0                 100.0   
Wales                 64.0           0.0         0.0                   0.0   

term              Carlo Ancelotti  ...  Sara Cox   Scotland  Sloveni

In [49]:
basket_encoded_UK = basket_UK.map(hot_encode) 
basket_UK = basket_encoded_UK

print(basket_UK)

term              Alan Shearer  Albania  Apple  BGT final  Bath Rugby  \
region_name                                                             
England                      0        0      1          1           1   
Northern Ireland             1        0      1          0           0   
Scotland                     0        0      1          0           1   
Wales                        1        1      1          0           1   

term              Billings  Billy Harris  Bridgerton  Britain's Got Talent  \
region_name                                                                  
England                  0             1           0                     0   
Northern Ireland         1             0           0                     0   
Scotland                 0             0           1                     1   
Wales                    1             0           0                     0   

term              Carlo Ancelotti  ...  Sara Cox  Scotland  Slovenia  Spain  \
region_name  

In [50]:
# Building the model 
frq_items = apriori(basket_UK.astype('bool'), min_support = 0.8, use_colnames = True)
  
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "United_Kingdom")

antecedents                                        consequents  \
0               (Apple)                                 (England football)   
1    (England football)                                            (Apple)   
2               (Apple)                                     (Gary Glitter)   
3        (Gary Glitter)                                            (Apple)   
4               (Apple)                                      (Love Island)   
..                  ...                                                ...   
597  (England football)       (UFC, Apple, Gary Glitter, Love Island, UCL)   
598             (Apple)  (UFC, England football, Gary Glitter, Love Isl...   
599      (Gary Glitter)   (UFC, England football, Apple, Love Island, UCL)   
600       (Love Island)  (UFC, England football, Apple, Gary Glitter, UCL)   
601               (UCL)  (UFC, England football, Apple, Gary Glitter, L...   

     antecedent support  consequent support  support  confidence  lift  \
0                   1.0                 1.0      1.0         1.0   1.0   
1                   1.0                 1.0      1.0         1.0   1.0   
2                   1.0                 1.0      1.0         1.0   1.0   
3                   1.0                 1.0      1.0         1.0   1.0   
4                   1.0                 1.0      1.0         1.0   1.0   
..                  ...                 ...      ...         ...   ...   
597                 1.0                 1.0      1.0         1.0   1.0   
598                 1.0                 1.0      1.0         1.0   1.0   
599                 1.0                 1.0      1.0         1.0   1.0   
600                 1.0                 1.0      1.0         1.0   1.0   
601                 1.0                 1.0      1.0         1.0   1.0   

     leverage  conviction  zhangs_metric  
0         0.0         inf            0.0  
1         0.0         inf            0.0  
2         0.0         inf            0.0  
3         0.0         inf            0.0  
4         0.0         inf            0.0  
..        ...         ...            ...  
597       0.0         inf            0.0  
598       0.0         inf            0.0  
599       0.0         inf            0.0  
600       0.0         inf            0.0  
601       0.0         inf            0.0  

[602 rows x 10 columns]

In [51]:
# Top terms in South Africa
basket_SF = (df[df['country_name'] == "South Africa"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_SF)


term               Apple  Argentina vs Canada  Bangladesh vs Netherlands  \
region_name                                                                
Eastern Cape   62.692308                  0.0                        0.0   
Free State     60.250000                 81.0                        0.0   
Gauteng        64.727273                  0.0                        0.0   
KwaZulu-Natal  64.923077                  0.0                        0.0   
Limpopo        59.074074                  0.0                        0.0   
Mpumalanga     58.285714                  0.0                       85.0   
North West     61.071429                  0.0                        0.0   
Northern Cape  63.500000                  0.0                        0.0   
Western Cape   69.081081                  0.0                        0.0   

term           Baroka FC  Bridgerton  Charley Hull  Coco Gauff  \
region_name                                                      
Eastern Cape   51.000000   87.0

In [52]:
basket_encoded_SF = basket_SF.map(hot_encode) 
basket_SF = basket_encoded_SF

print(basket_SF)

term           Apple  Argentina vs Canada  Bangladesh vs Netherlands  \
region_name                                                            
Eastern Cape       1                    0                          0   
Free State         1                    1                          0   
Gauteng            1                    0                          0   
KwaZulu-Natal      1                    0                          0   
Limpopo            1                    0                          0   
Mpumalanga         1                    0                          1   
North West         1                    0                          0   
Northern Cape      1                    0                          0   
Western Cape       1                    0                          0   

term           Baroka FC  Bridgerton  Charley Hull  Coco Gauff  \
region_name                                                      
Eastern Cape           1           1             0           0   
Free Stat

In [53]:
# Building the model 
frq_items = apriori(basket_SF.astype('bool'), min_support = 0.5, use_colnames = True)
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "South_Africa")

antecedents  \
434          (Cricket World Cup, Malawi)   
439                            (Lesotho)   
446          (Poetry, Cricket World Cup)   
451                            (Lesotho)   
844   (Apple, Cricket World Cup, Malawi)   
...                                  ...   
4671                             (Apple)   
4734                             (Apple)   
4795                             (Apple)   
5044                             (Apple)   
5171                             (Apple)   

                                            consequents  antecedent support  \
434                                           (Lesotho)            0.555556   
439                         (Cricket World Cup, Malawi)            0.555556   
446                                           (Lesotho)            0.555556   
451                         (Poetry, Cricket World Cup)            0.555556   
844                                           (Lesotho)            0.555556   
...                                                 ...                 ...   
4671  (Poetry, Orlando Pirates, Malawi, Lesotho, Sou...            1.000000   
4734  (T20 World Cup, Poetry, Orlando Pirates, Malaw...            1.000000   
4795  (Poetry, The Boys, Orlando Pirates, Malawi, So...            1.000000   
5044  (Poetry, Cricket World Cup, Orlando Pirates, M...            1.000000   
5171  (T20 World Cup, Poetry, Orlando Pirates, Malaw...            1.000000   

      consequent support   support  confidence  lift  leverage  conviction  \
434             0.555556  0.555556    1.000000   1.8  0.246914         inf   
439             0.555556  0.555556    1.000000   1.8  0.246914         inf   
446             0.555556  0.555556    1.000000   1.8  0.246914         inf   
451             0.555556  0.555556    1.000000   1.8  0.246914         inf   
844             0.555556  0.555556    1.000000   1.8  0.246914         inf   
...                  ...       ...         ...   ...       ...         ...   
4671            0.555556  0.555556    0.555556   1.0  0.000000         1.0   
4734            0.555556  0.555556    0.555556   1.0  0.000000         1.0   
4795            0.555556  0.555556    0.555556   1.0  0.000000         1.0   
5044            0.555556  0.555556    0.555556   1.0  0.000000         1.0   
5171            0.555556  0.555556    0.555556   1.0  0.000000         1.0   

      zhangs_metric  
434             1.0  
439             1.0  
446             1.0  
451             1.0  
844             1.0  
...             ...  
4671            0.0  
4734            0.0  
4795            0.0  
5044            0.0  
5171            0.0  

[5176 rows x 10 columns]

In [54]:
# Top terms in Canada
basket_Canada = (df[df['country_name'] == "Canada"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Canada)

term                       Alex Caruso   Billings  Bobrovsky  Bridgerton  \
region_name                                                                
Alberta                          100.0  56.500000        0.0         0.0   
British Columbia                   0.0   0.000000      100.0        57.0   
Manitoba                           0.0  84.000000        0.0         0.0   
New Brunswick                      0.0   0.000000        0.0         0.0   
Newfoundland and Labrador          0.0   0.000000        0.0        76.5   
Northwest Territories              0.0   0.000000        0.0         0.0   
Nova Scotia                        0.0   0.000000        0.0        71.0   
Nunavut                            0.0   0.000000      100.0         0.0   
Ontario                           54.0  68.400000       86.0         0.0   
Prince Edward Island               0.0   0.000000       94.0        71.5   
Québec                            63.0  58.363636        0.0        85.0   
Saskatchewan

In [55]:
basket_encoded_Canada = basket_Canada.map(hot_encode) 
basket_Canada = basket_encoded_Canada

print(basket_Canada)

term                       Alex Caruso  Billings  Bobrovsky  Bridgerton  \
region_name                                                               
Alberta                              1         1          0           0   
British Columbia                     0         0          1           1   
Manitoba                             0         1          0           0   
New Brunswick                        0         0          0           0   
Newfoundland and Labrador            0         0          0           1   
Northwest Territories                0         0          0           0   
Nova Scotia                          0         0          0           1   
Nunavut                              0         0          1           0   
Ontario                              1         1          1           0   
Prince Edward Island                 0         0          1           1   
Québec                               1         1          0           1   
Saskatchewan             

In [56]:
# Building the model 
frq_items = apriori(basket_Canada.astype('bool'), min_support = 0.4, use_colnames = True)
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Canada")

antecedents                       consequents  \
479          (PGA, Buffalo)                  (LPGA, The Boys)   
704      (Oilers, The Boys)                    (PGA, Rangers)   
752      (Et, PGA, Buffalo)                  (The Boys, LPGA)   
763          (PGA, Buffalo)              (The Boys, Et, LPGA)   
805  (Et, Oilers, The Boys)                    (PGA, Rangers)   
..                      ...                               ...   
664                    (Et)       (Oilers, Rangers, The Boys)   
737                    (Et)  (Oilers, Buffalo, Rangers, LPGA)   
766                    (Et)    (The Boys, PGA, Buffalo, LPGA)   
796                    (Et)    (The Boys, PGA, Rangers, LPGA)   
826                    (Et)  (PGA, Oilers, Rangers, The Boys)   

     antecedent support  consequent support   support  confidence      lift  \
479            0.461538            0.538462  0.461538    1.000000  1.857143   
704            0.461538            0.538462  0.461538    1.000000  1.857143   
752            0.461538            0.538462  0.461538    1.000000  1.857143   
763            0.461538            0.538462  0.461538    1.000000  1.857143   
805            0.461538            0.538462  0.461538    1.000000  1.857143   
..                  ...                 ...       ...         ...       ...   
664            1.000000            0.461538  0.461538    0.461538  1.000000   
737            1.000000            0.461538  0.461538    0.461538  1.000000   
766            1.000000            0.461538  0.461538    0.461538  1.000000   
796            1.000000            0.461538  0.461538    0.461538  1.000000   
826            1.000000            0.461538  0.461538    0.461538  1.000000   

     leverage  conviction  zhangs_metric  
479  0.213018         inf       0.857143  
704  0.213018         inf       0.857143  
752  0.213018         inf       0.857143  
763  0.213018         inf       0.857143  
805  0.213018         inf       0.857143  
..        ...         ...            ...  
664  0.000000         1.0       0.000000  
737  0.000000         1.0       0.000000  
766  0.000000         1.0       0.000000  
796  0.000000         1.0       0.000000  
826  0.000000         1.0       0.000000  

[830 rows x 10 columns]